In [ ]:
"""
Created on Thu Feb 08 10:42 2024

Look at some 2D variables in the Casimir runs

Author: @claraburgard
"""

In [1]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import cmocean

import cartopy.crs as ccrs
import cartopy.feature
from cartopy.util import add_cyclic_point
import seaborn as sns
import cartopy
import matplotlib.colors as colors
from matplotlib.colors import Normalize
from cartopy.feature import LAND

import gsw

from basal_melt_NEMO.constants import *
import basal_melt_NEMO.figure_functions as figf
import basal_melt_NEMO.useful_functions as uf

In [2]:
sns.set_context('paper')

In [3]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


READ IN DATA

In [4]:
inputpath_closed='/data/cdelaver/n42tm21/'
inputpath_open='/data/cdelaver/n42openc/'
clara_path='/data/cburgard/CASIMIR_SIMU/interim/XR_PROCESSED/'
plot_path = '/data/cburgard/PLOTS/first_plots/'
mask_path = '/data/cburgard/TOOLS/'

In [5]:
ocean_masks = xr.open_dataset(mask_path + 'basin_masks_orca1_nemo4p2.nc')

In [6]:
file_open_mean10 = xr.open_mfdataset(inputpath_open + 'n42openc_00910101_01001231_1Y_grid_T.nc').mean('time_counter')
file_closed_mean10 = xr.open_mfdataset(inputpath_closed + 'n42tm21_00910101_01001231_1Y_grid_T.nc').mean('time_counter')

file_open_V_mean10 = xr.open_mfdataset(inputpath_open + 'n42openc_00910101_01001231_1Y_grid_V.nc').mean('time_counter')
file_closed_V_mean10 = xr.open_mfdataset(inputpath_closed + 'n42tm21_00910101_01001231_1Y_grid_V.nc').mean('time_counter')

In [7]:
file_open_T = xr.open_mfdataset(inputpath_open + 'n42openc_00910101_01001231_1Y_grid_T.nc')
file_closed_T = xr.open_mfdataset(inputpath_closed + 'n42tm21_00910101_01001231_1Y_grid_T.nc')

file_open_V = xr.open_mfdataset(inputpath_open + 'n42openc_00910101_01001231_1Y_grid_V.nc')
file_closed_V = xr.open_mfdataset(inputpath_closed + 'n42tm21_00910101_01001231_1Y_grid_V.nc')

In [8]:
file_open_mean10 = file_open_mean10.where(file_open_mean10['so'] > 0)
file_open_V_mean10 = file_open_V_mean10#.where(file_open_mean10['so'].rename({'deptht':'depthv'}) > 0)

file_open_T = file_open_T.where(file_open_T['so'] > 0)

In [9]:
lon = file_open_mean10.nav_lon
lat = file_open_mean10.nav_lat

In [ ]:
T_30W_closed = file_closed_mean10['thetao'].where((lon > -31.) & (lon < -29.)).mean('x')
T_30W_open = file_open_mean10['thetao'].where((lon > -31.) & (lon < -29.)).mean('x')

In [ ]:
# checking if the line is right
#ref = file_closed_mean10['thetao'].where((lon > -31.) & (lon < -29.)).isel(deptht=0)
ref = ocean_masks['pacific']
plt.figure()

llon = lon
llat = lat

proj = ccrs.PlateCarree(central_longitude=0)
wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

In [10]:
def compute_density(da):
    depth = da['e3t'].cumsum('deptht')
    S = da['so']
    T = da['thetao']
    return gsw.rho(S,T,depth)

In [11]:
def compute_sigma2(da):
    S = da['so']
    T = da['thetao']
    return gsw.density.sigma2(S, T)

In [12]:
def transect_along_ocean(var, lon, lon_min, lon_max, lat, lat_max, cpalette, varname):

    if lat_max == 90:
        plot_var_closed = file_closed_mean10[var].where((lon > lon_min) & (lon < lon_max) & (lat <= lat_max)).mean('x')
        plot_var_open = file_open_mean10[var].where((lon > lon_min) & (lon < lon_max) & (lat <= lat_max)).mean('x')
    else:
        plot_var_closed = file_closed_mean10[var].where((lon > lon_min) & (lon < lon_max) & (lat <= lat_max), drop=True).mean('x')
        plot_var_open = file_open_mean10[var].where((lon > lon_min) & (lon < lon_max) & (lat <= lat_max), drop=True).mean('x')
    
    f, ax = plt.subplots(3, 1, sharex=True, sharey=True, figsize=(8.25,8.25*3))

    abs1 = ax[0].contourf(plot_var_closed.y,-1*plot_var_closed.deptht, plot_var_closed, cmap=cpalette)
    f.colorbar(abs1, ax=ax[0], orientation='vertical')
    ax[0].set_title(varname)
    
    abs2 = ax[1].contourf(plot_var_open.y,-1*plot_var_open.deptht, plot_var_open, cmap=cpalette)
    f.colorbar(abs2, ax=ax[1], orientation='vertical')
    
    ax_limit = (np.abs(plot_var_open - plot_var_closed)).max().values
    abs3 = ax[2].contourf(plot_var_open.y,-1*plot_var_open.deptht, (plot_var_open - plot_var_closed), cmap=mpl.cm.coolwarm, vmin=-ax_limit, vmax=ax_limit)
    f.colorbar(abs3, ax=ax[2], orientation='vertical')

In [13]:
def streamfunction_along_ocean(vo_open, vo_closed, ocean, cpalette):

    if ocean == 'indo-pacific':
        vmass_sum_open = vo_open.where(np.isfinite(ocean_masks['indian']) | np.isfinite(ocean_masks['pacific'])).sum('x') / 10**6
        vmass_sum_closed = vo_closed.where(np.isfinite(ocean_masks['indian']) | np.isfinite(ocean_masks['pacific'])).sum('x') / 10**6
    else:
        vmass_sum_open = vo_open.where(np.isfinite(ocean_masks[ocean])).sum('x') / 10**6
        vmass_sum_closed = vo_closed.where(np.isfinite(ocean_masks[ocean])).sum('x') / 10**6

    vmass_sum_open = vmass_sum_open.sel(depthv=vmass_sum_open.depthv[::-1])
    vmass_sum_closed = vmass_sum_closed.sel(depthv=vmass_sum_closed.depthv[::-1])

    plot_var_open = -1*vmass_sum_open.cumsum('depthv')
    plot_var_closed = -1*vmass_sum_closed.cumsum('depthv')
    
    f, ax = plt.subplots(3, 1, sharex=True, sharey=True, figsize=(8.25,8.25*3))

    ax_limit0 = 20 #np.quantile(np.array([np.abs(plot_var_open),np.abs(plot_var_closed)]), 0.99)
    
    abs1 = ax[0].pcolormesh(plot_var_closed.y,-1*plot_var_closed.depthv, plot_var_closed, cmap=cpalette, vmin=-ax_limit0, vmax=ax_limit0)
    f.colorbar(abs1, ax=ax[0], orientation='vertical')
    ax[0].set_title('Streamfunction '+ocean)

    abs2 = ax[1].pcolormesh(plot_var_open.y,-1*plot_var_open.depthv, plot_var_open, cmap=cpalette, vmin=-ax_limit0, vmax=ax_limit0)
    f.colorbar(abs2, ax=ax[1], orientation='vertical')
    
    ax_limit = (np.abs(plot_var_open - plot_var_closed)).max().values
    abs3 = ax[2].pcolormesh(plot_var_open.y,-1*plot_var_open.depthv, (plot_var_open - plot_var_closed), cmap=mpl.cm.coolwarm, vmin=-ax_limit, vmax=ax_limit)
    f.colorbar(abs3, ax=ax[2], orientation='vertical')

In [ ]:
### ATLANTIC

In [ ]:
f_temp = transect_along_ocean('thetao', lon, -31., -29., lat, 90, cmocean.cm.thermal,'Conservative Temperature [°C]')

In [ ]:
f_sal = transect_along_ocean('so', lon, -31., -29., lat, 90,cmocean.cm.haline,'Absolute Salinity [g/kg]')

In [ ]:
file_closed_mean10['rho'] = compute_density(file_closed_mean10).load()
file_open_mean10['rho'] = compute_density(file_open_mean10).load()

In [ ]:
f_rho = transect_along_ocean('rho', lon, -31., -29., lat, 90, cmocean.cm.dense,'In-situ density [kg/m3]')

In [ ]:
file_closed_mean10['sigma2'] = compute_sigma2(file_closed_mean10).load()
file_open_mean10['sigma2'] = compute_sigma2(file_open_mean10).load()

In [ ]:
f_sigma2 = transect_along_ocean('sigma2', lon, -31., -29., lat, 90,cmocean.cm.dense,'Potential density anomaly with reference pressure of 2000 dbar [kg/m3]')

In [ ]:
### PACIFIC

In [ ]:
f_temp = transect_along_ocean('thetao', lon, -171, -169.,lat, 90,cmocean.cm.thermal,'Conservative Temperature [°C]')

In [ ]:
f_sal = transect_along_ocean('so', lon,  -171, -169.,lat, 90,cmocean.cm.haline,'Absolute Salinity [g/kg]')

In [ ]:
f_rho = transect_along_ocean('rho', lon, -171, -169.,lat, 90,cmocean.cm.dense,'In-situ density [kg/m3]')

In [ ]:
f_sigma2 = transect_along_ocean('sigma2', lon, -171, -169.,lat, 90,cmocean.cm.dense,'Potential density anomaly with reference pressure of 2000 dbar [kg/m3]')

In [ ]:
### WEDDELL SEA

In [ ]:
f_temp = transect_along_ocean('thetao', lon, -51, -49,lat, -50,cmocean.cm.thermal,'Conservative Temperature [°C] Weddell Sea')

In [ ]:
f_sal = transect_along_ocean('so', lon,  -51, -49,lat,  -50,cmocean.cm.haline,'Absolute Salinity [g/kg] Weddell Sea')

In [ ]:
f_sigma2 = transect_along_ocean('sigma2', lon, -51, -49,lat, -50,cmocean.cm.dense,'Potential density anomaly with reference pressure of 2000 dbar [kg/m3] Weddell Sea')

In [ ]:
### ROSS SEA

In [ ]:
f_temp = transect_along_ocean('thetao', lon, -180, -179,lat, -50,cmocean.cm.thermal,'Conservative Temperature [°C] Ross Sea')

In [ ]:
f_sal = transect_along_ocean('so', lon,  -180, -179,lat,  -50,cmocean.cm.haline,'Absolute Salinity [g/kg] Ross Sea')

In [ ]:
f_sigma2 = transect_along_ocean('sigma2', lon, -180, -179,lat, -50,cmocean.cm.dense,'Potential density anomaly with reference pressure of 2000 dbar [kg/m3] Ross Sea')

In [ ]:
mask_Ross = ((lon <= -120) | (lon >= 150)) & (lat >= -87.) & (lat <= -65.)

In [ ]:
### STREAM FUNCTION

In [ ]:
streamfunction_along_ocean(file_open_V_mean10['vocetr_eff'],file_closed_V_mean10['vocetr_eff'],'indo-pacific',mpl.cm.coolwarm)

In [ ]:
streamfunction_along_ocean(file_open_V_mean10['vocetr_eff'],file_closed_V_mean10['vocetr_eff'],'atlantic',mpl.cm.coolwarm)

In [ ]:
streamfunction_along_ocean(file_open_V_mean10['vocetr_eff'],file_closed_V_mean10['vocetr_eff'],'global',mpl.cm.coolwarm)

In [ ]:
#### BINNED STREAM FUNCTION

In [ ]:
### Compute potential density anomaly with reference pressure of 2000 dbar

In [14]:
file_closed_T['sigma2'] = compute_sigma2(file_closed_T)
file_open_T['sigma2'] = compute_sigma2(file_open_T)

In [ ]:
print(file_closed_T['sigma2'].max().load()) #37.8069389
print(file_closed_T['sigma2'].min().load()) #10.16092161

In [ ]:
print(file_open_T['sigma2'].max().load()) #37.80702688
print(file_open_T['sigma2'].min().load()) #9.813129

In [ ]:
### Interpolate from V to T grid

In [15]:
sigma2_V_closed = (file_closed_T['sigma2'] + file_closed_T['sigma2'].shift(y=-1)) / 2
sigma2_V_closed.loc[{'y': sigma2_V_closed.y.max()}] = file_closed_T['sigma2'].sel(y=sigma2_V_closed.y.max())

sigma2_V_open = (file_open_T['sigma2'] + file_open_T['sigma2'].shift(y=-1)) / 2
sigma2_V_open.loc[{'y': sigma2_V_open.y.max()}] = file_open_T['sigma2'].sel(y=sigma2_V_open.y.max())

In [16]:
vtra_closed = file_closed_V['vocetr_eff']
sigma2_V_closed = sigma2_V_closed.rename({'deptht': 'depthv'}).where(np.isfinite(vtra_closed))

vtra_open = file_open_V['vocetr_eff']
sigma2_V_open = sigma2_V_open.rename({'deptht': 'depthv'}).where(np.isfinite(vtra_open))

In [18]:
print(sigma2_V_closed.max().load()) #37.80693871
print(sigma2_V_closed.min().load()) #10.23064935

<xarray.DataArray 'sigma2' ()>
array(37.80693871)
<xarray.DataArray 'sigma2' ()>
array(10.23064935)


In [19]:
print(sigma2_V_open.max().load()) #37.80702652
print(sigma2_V_open.min().load()) #9.82776096

<xarray.DataArray 'sigma2' ()>
array(37.80702652)
<xarray.DataArray 'sigma2' ()>
array(9.82776096)


In [ ]:
### Compute cell volume, depth, bathy

In [22]:
volu_closed = file_closed_V.e3v * file_closed_V.area
volu_open = file_open_V.e3v * file_open_V.area


In [24]:
print((volu_closed.sum()/10).load()) #1.363346e+18
print((volu_open.sum()/10).load()) #2.6498255e+18

<xarray.DataArray ()>
array(1.363346e+18, dtype=float32)
<xarray.DataArray ()>
array(2.6498255e+18, dtype=float32)


In [25]:
e3v_closed = file_closed_V.e3v.where(np.isfinite(vtra_closed))
e3v_open = file_open_V.e3v.where(np.isfinite(vtra_open))

In [31]:
e3v_closed

<xarray.DataArray 'e3v' (time_counter: 10, depthv: 75, y: 331, x: 360)>
dask.array<where, shape=(10, 75, 331, 360), dtype=float32, chunksize=(10, 75, 331, 360), chunktype=numpy.ndarray>
Coordinates:
    nav_lat        (y, x) float32 dask.array<chunksize=(331, 360), meta=np.ndarray>
    nav_lon        (y, x) float32 dask.array<chunksize=(331, 360), meta=np.ndarray>
  * depthv         (depthv) float32 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
    time_centered  (time_counter) object dask.array<chunksize=(10,), meta=np.ndarray>
  * time_counter   (time_counter) object 0091-07-02 12:00:00 ... 0100-07-02 1...
Dimensions without coordinates: y, x
Attributes:
    standard_name:       cell_thickness
    long_name:           V-cell thickness
    units:               m
    online_operation:    average
    interval_operation:  2700 s
    interval_write:      1 yr
    cell_methods:        time: mean (interval: 2700 s)
    cell_measures:       area: area

In [50]:
depth_closed = xr.concat([xr.DataArray(data=np.zeros((len(e3v_closed.time_counter),len(e3v_closed.y),len(e3v_closed.x))), dims=['time_counter','y','x']).assign_coords(
    {'time_counter': e3v_closed.time_counter, 'y': e3v_closed.y, 'x': e3v_closed.x, 'depthv': 0}),
    e3v_closed.cumsum('depthv')], dim='depthv')
depth_closed = depth_closed.where(np.isfinite(e3v_closed))

depth_open = xr.concat([xr.DataArray(data=np.zeros((len(e3v_open.time_counter),len(e3v_open.y),len(e3v_open.x))), dims=['time_counter','y','x']).assign_coords(
    {'time_counter': e3v_open.time_counter, 'y': e3v_open.y, 'x': e3v_open.x, 'depthv': 0}),
    e3v_open.cumsum('depthv')], dim='depthv')
depth_open = depth_open.where(np.isfinite(e3v_open))

In [47]:
depth_mid_closed = (depth_closed + depth_closed .shift(depthv=-1)) / 2
depth_mid_open = (depth_open + depth_open .shift(depthv=-1)) / 2

In [63]:
bathy_closed = depth_closed.isel(time_counter=9).max('depthv')
bathy_open = depth_open.isel(time_counter=9).max('depthv')

In [64]:
### double check on the open stuff => needs a mask I guess!

bathy_open.plot()

In [53]:
bathy = np.isfinite(depth_closed)

<xarray.DataArray (time_counter: 10, y: 331, x: 360, depthv: 76)>
dask.array<truediv, shape=(10, 331, 360, 76), dtype=float64, chunksize=(10, 331, 360, 75), chunktype=numpy.ndarray>
Coordinates:
    nav_lat        (y, x) float32 dask.array<chunksize=(331, 360), meta=np.ndarray>
    nav_lon        (y, x) float32 dask.array<chunksize=(331, 360), meta=np.ndarray>
    time_centered  (time_counter) object dask.array<chunksize=(10,), meta=np.ndarray>
  * time_counter   (time_counter) object 0091-07-02 12:00:00 ... 0100-07-02 1...
  * y              (y) int64 0 1 2 3 4 5 6 7 ... 323 324 325 326 327 328 329 330
  * x              (x) int64 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
  * depthv         (depthv) float64 0.0 0.5058 1.556 ... 5.698e+03 5.902e+03

In [52]:
depth_open.isel(time_counter=0,y=100,x=100).load()

<xarray.DataArray (depthv: 75)>
array([1.02368915e+00, 2.10274315e+00, 3.25061846e+00, 4.48409986e+00,
       5.82400036e+00, 7.29589319e+00, 8.93078804e+00, 1.07656097e+01,
       1.28432636e+01, 1.52120380e+01, 1.79241161e+01, 2.10331039e+01,
       2.45907650e+01, 2.86435661e+01, 3.32299042e+01, 3.83789482e+01,
       4.41116371e+01, 5.04437523e+01, 5.73903732e+01, 6.49708023e+01,
       7.32131500e+01, 8.21581116e+01, 9.18619003e+01, 1.02398430e+02,
       1.13861061e+02, 1.26364098e+02, 1.40044250e+02, 1.55062119e+02,
       1.71603806e+02, 1.89882462e+02, 2.10139832e+02, 2.32647568e+02,
       2.57708130e+02, 2.85655121e+02, 3.16852570e+02, 3.51693329e+02,
       3.90595642e+02, 4.33998352e+02, 4.82353882e+02, 5.36119324e+02,
       5.95745544e+02, 6.61664673e+02, 7.34276184e+02, 8.13932800e+02,
       9.00926636e+02, 9.95477051e+02, 1.09772095e+03, 1.20770654e+03,
       1.32539099e+03, 1.45064246e+03, 1.58324609e+03, 1.72291382e+03,
       1.86929602e+03, 2.02199536e+03, 2.18058105e+03, 2.34460278e+03,
       2.51360303e+03, 2.68712842e+03, 2.86473828e+03, 3.04601196e+03,
       3.23055396e+03, 3.41799658e+03, 3.60800244e+03, 3.80026440e+03,
       3.99450513e+03, 4.19047656e+03, 4.38795703e+03, 4.58675049e+03,
       4.78668457e+03, 4.98760742e+03, 5.18938574e+03, 5.35585889e+03,
       5.52237061e+03, 5.72612158e+03, 5.92987256e+03])
Coordinates:
    time_counter   object 0091-07-02 12:00:00
    y              int64 100
    x              int64 100
  * depthv         (depthv) float64 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
    nav_lat        float32 -57.43
    nav_lon        float32 173.5
    time_centered  object 0091-07-02 12:00:00

In [ ]:
print((volu_closed.sum()/10).load()) #1.363346e+18
print((volu_open.sum()/10).load()) #2.6498255e+18

In [ ]:
file_open_V_mean10['vocetr_eff'].sum('x').plot()

In [ ]:
v_atlsum_open = file_open_V_mean10['vocetr_eff'].where(np.isfinite(ocean_masks['global'])).sum('x') / 10**6
v_atlsum_closed = file_closed_V_mean10['vocetr_eff'].where(np.isfinite(ocean_masks['global'])).sum('x') / 10**6

In [ ]:
v_atlsum_open_rev = v_atlsum_open.sel(depthv=v_atlsum_open.depthv[::-1])
v_atlsum_closed_rev = v_atlsum_closed.sel(depthv=v_atlsum_closed.depthv[::-1])

In [ ]:
v_atlsum_closed_rev.plot()

In [ ]:
streamf_atl_open = v_atlsum_open.sel(depthv=v_atlsum_open.depthv[::-1]).cumsum('depthv')
streamf_atl_closed = v_atlsum_closed.sel(depthv=v_atlsum_closed.depthv[::-1]).cumsum('depthv')

In [ ]:
streamf_atl_closed.plot()

In [ ]:
streamf_atl_closed.plot(vmax=20)

In [ ]:
streamf_atl_open_rev = v_atlsum_open_rev.cumsum('depthv')
streamf_atl_closed_rev = v_atlsum_closed_rev.cumsum('depthv').assign_coords({'depthv': -1*v_atlsum_closed_rev.depthv})


In [ ]:
plt.figure()
streamf_atl_closed_rev.plot(vmax=20)

In [ ]:
streamf_atl_open.plot()

In [ ]:
file_open_V_mean10['vocetr_eff'].where(np.isfinite(ocean_masks['atlantic'])).isel(depthv=0).plot()

In [ ]:
v_atlsum_open.plot()

In [ ]:
v_atlsum_open.cumsum('depthv').plot()

In [ ]:
plt.figure()
v_atlsum_open_rev.cumsum('depthv').plot()